In [4]:
# 导入模块
import os # 用于操作系统的交互，例如获取环境变量。
import openai # OpenAI 的官方库，用于与 OpenAI 服务进行交互。
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY'] # 从环境变量中获取 OPENAI_API_KEY 作为 API 密钥。
openai.api_type = "azure"
openai.api_base = "https://zjx.openai.azure.com/"
openai.api_version = "2023-05-15"

In [5]:
# 定义了一个名为 get_completion 的函数，其主要目的是使用 OpenAI 的 API 为给定的 prompt 获取模型的回复
def get_completion(prompt):
    # messages 是一个列表，其中包含一个字典，这个字典定义了一个用户角色和他们的消息内容。
    # 这是 OpenAI 的 Chat API 接受的格式，它允许多次交互（例如，先由用户发送消息，然后由模型回复，然后再由用户发送消息，等等）
    messages = [{"role": "user", "content": prompt}] 
    # 使用 try 和 except 来尝试执行某些代码，并在出现异常时捕获该异常。
    try:
        response = openai.ChatCompletion.create(
            engine="gpt35",  # 指定使用哪个模型。在这里，它是 "gpt35"。
            messages=messages,
            temperature=0, # 控制输出随机性的参数。值为 0 表示模型会产生最确定性的输出；较高的值（例如 1.0）会产生更多的随机性。
        )
        return response.choices[0].message["content"] # 从模型的响应中获取其消息内容并返回。
    except Exception as e:
        print(f"Error: {e}")
        return None

In [6]:
## 在这里，max_tokens主要限制的是输出的令牌数量，也就是模型生成的内容的最大长度。
def get_completion_from_messages(messages, 
                                 max_tokens=500):
    try:
        response = openai.ChatCompletion.create(
            max_tokens=max_tokens,
            engine="gpt35",
            messages=messages,
            temperature=0,
        )
    except Exception as e:
        print(f"Error: {e}")
    return response.choices[0].message["content"]

In [10]:
# 模型的局限性,即使模型在其训练过程中接触了大量的知识，它也没有完美地记住它所看到的信息。
# 这意味着它可能会尝试回答一些关于不常见主题的问题，并可能编造出听起来合理但实际上不真实的答案。
# 这种情况被称为“幻觉”。
# 为了减少幻觉，可以要求模型首先从文本中找到任何相关的引文，然后使用这些引文来回答问题。

user_message = f"""
太阳系简介

太阳系由一个中心恒星——太阳和围绕其旋转的八大行星组成。
最靠近太阳的是水星，一个炎热的、没有大气层的小行星。接下来是金星，一个有着浓厚大气层和火山活动的行星。
地球是第三颗行星，是我们的家园，拥有丰富的生命和水资源。火星，被称为红色行星，有着薄薄的大气层和冰冻的水。然后是巨大的木星，拥有最多的卫星。
紧随其后的是带有显著的环的土星。天王星和海王星是两颗冰冷的巨大气态行星，位于太阳系的边缘。

请从文章中找到与火星和水相关的任何引文，然后根据上述引文，火星上有水吗？

"""

messages =  [   
{'role':'user', 
 'content': f"{user_message}"},  
] 

response = get_completion_from_messages(messages)
print(response)

"火星，被称为红色行星，有着薄薄的大气层和冰冻的水。" 根据这句话，火星上有冰冻的水，但没有明确提到液态水的存在。
